# Using just TensorFlow Framework for IRIS dataset

### Step 1: Import the required libraries


In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf

### Step 2: Load the `IRIS` dataset

In [2]:
path = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/iris.csv' #import IRIS dataset
df = pd.read_csv(path, header=None)

In [3]:
df.head(2)

,0,1,2,3,4
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa


In [4]:
headers = ["sepal_length", "sepal_width", "petal_length", "petal_width", "species"]
### Assign the headers to `df`
df.columns = headers

df.sample(2)

,sepal_length,sepal_width,petal_length,petal_width,species
115,6.4,3.2,5.3,2.3,Iris-virginica
142,5.8,2.7,5.1,1.9,Iris-virginica


In [5]:
### Import SKlearn libraries
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [6]:
### Let us prepare the data - separating the features from the target
X, y = df.values[:,:-1], df.values[:,-1]

In [7]:
X[:2]

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3.0, 1.4, 0.2]], dtype=object)

In [8]:
y[:2]

array(['Iris-setosa', 'Iris-setosa'], dtype=object)

In [9]:
X = X.astype('float32')

In [10]:
y = LabelEncoder().fit_transform(y)

In [11]:
y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

### Step 3: Split the `IRIS` dataset into `train` and `test`

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=.30, random_state=42)

In [13]:
X_train.shape, X_test.shape

((105, 4), (45, 4))

### Step 4: Build the neural network model

In [14]:
### Dertermine the number of features
n_features = X_train.shape[1]
print('Number of features:',n_features)

### Determine the number of classes
n_classes = len(np.unique(y))
print('Number of classes:', n_classes)

Number of features: 4
Number of classes: 3


#### Step 4A: Define the model

**`CNN:`Convoluted Neural Network**

*   initialize the weights and biases for each layer
*   implement the forward pass


In [15]:
class CNNModel(tf.Module):
  def __init__(self):#initializes weights and biases for each layer

    ### represents weight and bias for the first layer
    ### creating a TF variable 'w1' with dimensions: n_features, 20 neurons and initializing w1 with random values drawn
    ## from a normal distribution having std. deviaton of 0.1
    self.w1 = tf.Variable(tf.random.normal([n_features, 20], stddev=0.1), name='w1')
    self.b1 = tf.Variable(tf.zeros([20]), name='b1')

    ###  Initializing weights 'w2' and bias 'b2`while connecting 20 neurons from the first layer to 20 in the second layer
    self.w2 = tf.Variable(tf.random.normal([20, 20], stddev=0.1), name='w2')
    self.b2 = tf.Variable(tf.zeros([20]), name='b2')

    ###  Initializing weights 'w3' and bias 'b3`while connecting 20 neurons from the second layer to 15 in the third layer
    self.w3 = tf.Variable(tf.random.normal([20, 15], stddev=0.1), name='w3')
    self.b3 = tf.Variable(tf.zeros([15]), name='b3')

    ###  Initializing weights 'w4' and bias 'b4`while connecting 15 neurons from the third layer to 8 in the forth layer
    self.w4 = tf.Variable(tf.random.normal([15, 8], stddev=0.1), name='w4')
    self.b4 = tf.Variable(tf.zeros([8]), name='b4')

    ###  Initializing weights 'w5' and bias 'b5`while connecting 8 neurons from the forth layer to n_classes [3] in the output layer
    self.w5 = tf.Variable(tf.random.normal([8, n_classes], stddev=0.1), name='w5')
    self.b5 = tf.Variable(tf.zeros([n_classes]), name='b5')

  def __call__(self, x):# to implement the forward pass

    ### represents weight and bias for the first layer
    ### ReLU (REctified Linear Unit) - an activation function
    layer1 = tf.nn.relu(tf.add(tf.matmul(x,self.w1), self.b1)) # performing a matrix multiplication between input features and the weight matrix. Also adding the bias matrix after in the next using `tf.add`
    layer2 = tf.nn.relu(tf.add(tf.matmul(layer1,self.w2), self.b2))
    output = tf.add(tf.matmul(layer2,self.w3), self.b3)
    return output

#### Step 4B: Instantiate the model





In [16]:
model = CNNModel()

In [17]:
loss_fn = tf.losses.SparseCategoricalCrossentropy(from_logits=True) # Crossentropy loss for multi-class classification
optimizer = tf.optimizers.Adam() #Adam optimizer for training

#### Step 4C: Training Steps

In [18]:
### Training Function
@tf.function # compiles the function into a TensorFlow graph for efficiency
def train_step(x,y): #Performs one step of training
  with tf.GradientTape() as tape: #records operations for automatic differentiation
    logits = model(x)   # raw output of the model
    loss = loss_fn(y, logits) #compute the loss
  gradients = tape.gradient(loss, model.trainable_variables) #gradients of the loss w.r.t model parameters --> weights, biases
  optimizer.apply_gradients(zip(gradients,model.trainable_variables)) #updates the model parameters --> weights, biases
  return loss

#### Step 4D: Run the model loop

In [19]:
X_train.shape

(105, 4)

In [20]:
105/32

3.28125

In [21]:
n_epochs = 150 # no. of training epochs
batch_size = 32 # size of each training batch

for epoch in range(n_epochs): #looping over epochs
  dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(buffer_size=1000).batch(batch_size) # Creates a TF dataset from the training data,shuffles and bacthes it
  for batch_x, batch_y in dataset: #looping over batches
    loss = train_step(batch_x, batch_y) #refer the function train_step above
  if (epoch+1)%10 ==0:  #Print loss every 10 epochs
    print(f'Epoch {epoch + 1}, Loss: {loss.numpy()}')

/usr/local/lib/python3.10/dist-packages/keras/src/optimizers/base_optimizer.py:664: UserWarning: Gradients do not exist for variables ['b4:0', 'b5:0', 'w4:0', 'w5:0'] when minimizing the loss. If using `model.compile()`, did you forget to provide a `loss` argument?
  warnings.warn(


Epoch 10, Loss: 2.2768664360046387
Epoch 20, Loss: 1.2105457782745361
Epoch 30, Loss: 1.1089766025543213
Epoch 40, Loss: 0.9906724095344543
Epoch 50, Loss: 0.8448153734207153
Epoch 60, Loss: 0.7863245010375977
Epoch 70, Loss: 0.6381366848945618
Epoch 80, Loss: 0.6331114768981934
Epoch 90, Loss: 0.5072092413902283
Epoch 100, Loss: 0.4019402265548706
Epoch 110, Loss: 0.3890707194805145
Epoch 120, Loss: 0.3675156831741333
Epoch 130, Loss: 0.35778021812438965
Epoch 140, Loss: 0.2605552077293396
Epoch 150, Loss: 0.19671237468719482


In [23]:
def evaluate(X,y): #function to compute accuracy on the test set
  logits = model(X) # raw outputs of the model
  predictions = tf.argmax(logits, axis=1) #convers logits into predicted classes
  accuracy = np.mean(predictions.numpy()==y) #mean accuracy
  return accuracy

test_accuracy = evaluate(X_test, y_test)
print(f'Test Accuracy %: {test_accuracy*100:.3f}')

Test Accuracy %: 97.778
